In [1]:
import os
import cv2
import tensorflow as tf
import numpy as np
from functools import partial
from riptide.anneal.models.squeezenet import SqueezeNet
from riptide.anneal.models.get_models import get_model
from riptide.anneal.anneal_config import Config

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [3]:
cifar = tf.keras.datasets.cifar10.load_data()
(train_images, train_labels), (test_images, test_labels) = cifar

train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(train_images)

In [4]:
config = Config(quantize=False, a_bits=1, w_bits=1, fixed=False)
with config:
    model = SqueezeNet(classes=10)

model.compile(
    optimizer='adam',
    loss= 'categorical_crossentropy',
    metrics=['accuracy'])

test_input = tf.keras.layers.Input(shape=[32, 32, 3])
test_output = model(test_input)

model.summary()

Model: "squeeze_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 112, 112, 96)      14208     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 56, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 56, 56, 96)        384       
_________________________________________________________________
pact (PACT)                  (None, 56, 56, 96)        0         
_________________________________________________________________
sawb_conv2d (SAWBConv2D)     (None, 56, 56, 32)        3104      
_________________________________________________________________
batch_normalization_1 (Batch (None, 56, 56, 32)        

In [5]:
model.fit_generator(datagen.flow(train_images, train_labels, batch_size=32), steps_per_epoch=len(train_images)/32, epochs=10, validation_data=(test_images, test_labels))

Epoch 1/10
  16/1562 [..............................] - ETA: 53:12 - loss: 2.3028 - accuracy: 0.1365

KeyboardInterrupt: 